In [1]:
# Paso 0: Importar la libreria Pandas, Numpy y funciones de sklearn para trabajar con el modelo. 

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Paso 0: Aplicar '%ls' para identificar la carpeta en la que estamos trabajando.

%ls

 El volumen de la unidad C es Windows
 El número de serie del volumen es: 788B-24D1

 Directorio de C:\Users\javie\OneDrive\Escritorio\Ironhack\Semana 3\Banking-Risk\data\clean data

02/02/2024  11:32    <DIR>          .
02/02/2024  10:20    <DIR>          ..
02/02/2024  11:23    <DIR>          .ipynb_checkpoints
01/02/2024  16:26           147.594 1.1 - Cleaning dataset.ipynb
01/02/2024  17:07           123.363 1.2 - Analysis.ipynb
31/01/2024  18:41       166.133.370 application_data.csv
02/02/2024  11:32            21.158 Model 1.ipynb
02/02/2024  11:27        64.066.224 Model_1.csv
01/02/2024  15:44        51.924.049 risk_df.csv
02/02/2024  11:28            39.469 scoring_1.ipynb
01/02/2024  17:38            59.591 scoring_2.ipynb
02/02/2024  11:23                72 Untitled.ipynb
               9 archivos    282.514.890 bytes
               3 dirs  375.307.038.720 bytes libres


In [3]:
# Paso 1: Crear un nuevo Dataframe aplicando el metodo read_csv().

model_1 = pd.read_csv('Model_1.csv')

In [4]:
# Paso 2: Desplegar el Dataframe para comprobar la info con la que vamos a cosntruir la clasificación

model_1

,id,target,loan_type,gender,own_car,own_realty,children,client_income,credit_amount,loan_annuity,...,req_hour,req_day,req_week,req_mon,req_qrt,req_year,risk_category,risk_category_2,risk_category_3,risk_category_4
0,100002,High Risk,Cash loans,M,No,Yes,0,202500.0,406597.5,24700.5,...,0,0,0,0,0,1,High Risk,High Risk,High Risk,Low Risk
1,100003,Low Risk,Cash loans,F,No,No,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0,0,High Risk,High Risk,High Risk,High Risk
2,100004,Low Risk,Revolving loans,M,Yes,Yes,0,67500.0,135000.0,6750.0,...,0,0,0,0,0,0,High Risk,High Risk,High Risk,Low Risk
3,100006,Low Risk,Cash loans,F,No,Yes,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,2,High Risk,High Risk,High Risk,Low Risk
4,100007,Low Risk,Cash loans,M,No,Yes,0,121500.0,513000.0,21865.5,...,0,0,0,0,0,0,High Risk,High Risk,High Risk,High Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307500,456251,Low Risk,Cash loans,M,No,No,0,157500.0,254700.0,27558.0,...,0,0,0,0,0,2,High Risk,High Risk,Low Risk,Low Risk
307501,456252,Low Risk,Cash loans,F,No,Yes,0,72000.0,269550.0,12001.5,...,0,0,0,0,0,2,High Risk,High Risk,High Risk,High Risk
307502,456253,Low Risk,Cash loans,F,No,Yes,0,153000.0,677664.0,29979.0,...,1,0,0,1,0,1,High Risk,High Risk,High Risk,High Risk
307503,456254,High Risk,Cash loans,F,No,Yes,0,171000.0,370107.0,20205.0,...,0,0,0,0,0,0,High Risk,High Risk,High Risk,Low Risk


In [5]:
# Paso 3: Tras ver que la clasificaciones igual los valores, testear el nuevo Scoring de clasificación.

# Tabla de Contingencia

contingency_table = pd.crosstab(model_1['target'], model_1['risk_category_4'], margins=True)

print(contingency_table)

print()

# Matriz de confusión

conf_matrix = confusion_matrix(model_1['target'], model_1['risk_category_4'])

# Mostrar la matriz de confusión

print(conf_matrix)

print()

# Report de clasificación

print(classification_report(model_1['target'], model_1['risk_category_4']))

risk_category_4  High Risk  Low Risk     All
target                                      
High Risk            14756     10069   24825
Low Risk            168611    114069  282680
All                 183367    124138  307505

[[ 14756  10069]
 [168611 114069]]

              precision    recall  f1-score   support

   High Risk       0.08      0.59      0.14     24825
    Low Risk       0.92      0.40      0.56    282680

    accuracy                           0.42    307505
   macro avg       0.50      0.50      0.35    307505
weighted avg       0.85      0.42      0.53    307505



In [6]:
# Paso 4: Tras ver la fiabilidad del modelo, modificar los valores de la última clásificación del scoring.

condition_1 = model_1.client_income < (model_1.credit_amount / 3.5) # 3.5 en vez de 3.0
condition_2 = (model_1['ext_score_1'] > 0.7) & (model_1['ext_score_2'] > 0.7) & (model_1['ext_score_3'] > 0.7) # 0.7 en vez de 0.6
condition_3 = model_1.req_year > 5 # 5 en vez de 4

model_1['risk_category_5'] = np.select(
    [
        condition_1,
        condition_2,
        condition_3,
                    ],
    ['High Risk'] * 3,  
    default='Low Risk'   
)

In [7]:
# Paso 5: Testear el modelo con la modificación en las condiciones del scoring de 'High Risk'.

contingency_table = pd.crosstab(model_1['target'], model_1['risk_category_5'], margins=True)

print(contingency_table)

print()

conf_matrix = confusion_matrix(model_1['target'], model_1['risk_category_5'])

print(conf_matrix)

print()

print(classification_report(model_1['target'], model_1['risk_category_5']))

risk_category_5  High Risk  Low Risk     All
target                                      
High Risk            11997     12828   24825
Low Risk            137938    144742  282680
All                 149935    157570  307505

[[ 11997  12828]
 [137938 144742]]

              precision    recall  f1-score   support

   High Risk       0.08      0.48      0.14     24825
    Low Risk       0.92      0.51      0.66    282680

    accuracy                           0.51    307505
   macro avg       0.50      0.50      0.40    307505
weighted avg       0.85      0.51      0.62    307505



In [9]:
# Paso 6: Tras ver la fiabilidad del modelo, modificar nuevamente los valores de la última clásificación del scoring

condition_1 = (model_1.client_income < model_1.credit_amount / 4) # 4 en vez de 3.5 
condition_2 = (model_1['ext_score_1'] > 0.7) & (model_1['ext_score_2']> 0.7) & (model_1['ext_score_3']> 0.7)
condition_3 = model_1.req_year > 8 # 8 en vez de 5

model_1['risk_category_6'] = np.select(
    [
        condition_1,
        condition_2,
        condition_3,
                    ],
    ['High Risk'] * 3,  
    default='Low Risk'   
)

In [10]:
# Paso 7: Testear el modelo con la modificación en las condiciones del scoring de 'High Risk'.

contingency_table = pd.crosstab(model_1['target'], model_1['risk_category_6'], margins=True)

print(contingency_table)

print()

conf_matrix = confusion_matrix(model_1['target'], model_1['risk_category_6'])

print(conf_matrix)

print()

print(classification_report(model_1['target'], model_1['risk_category_6']))

risk_category_6  High Risk  Low Risk     All
target                                      
High Risk             9184     15641   24825
Low Risk            109980    172700  282680
All                 119164    188341  307505

[[  9184  15641]
 [109980 172700]]

              precision    recall  f1-score   support

   High Risk       0.08      0.37      0.13     24825
    Low Risk       0.92      0.61      0.73    282680

    accuracy                           0.59    307505
   macro avg       0.50      0.49      0.43    307505
weighted avg       0.85      0.59      0.68    307505

